<img src='imgs/cassandra.png' width='350'/>

In [1]:
!docker run -d                   \
    -v ${PWD}/cassandra_vol:/ddl \
    -h cassandra                 \
    -p 7000:7000                 \
    -p 127.0.0.1:9042:9042       \
    -p 127.0.0.1:9160:9160       \
    --name cassandra             \
    cassandra:3.11 

62b814ad1b6f34ffa97b5c78ac8dd996c596cec238bfd51c827695594ea3ad77


### check readiness

In [2]:
!docker exec -it cassandra /bin/bash -c "while ! cqlsh -e 'describe cluster' ; do sleep 5 ; done"
!echo "C* is ready"

Connection error: ('Unable to connect to any servers', {'127.0.0.1': error(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})
Connection error: ('Unable to connect to any servers', {'127.0.0.1': error(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})
Connection error: ('Unable to connect to any servers', {'127.0.0.1': error(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})

Cluster: Test Cluster
Partitioner: Murmur3Partitioner

C* is ready


### ddl

In [3]:
!docker exec -it cassandra cqlsh -e \
"CREATE KEYSPACE elk_c WITH replication = {'class': 'NetworkTopologyStrategy', 'datacenter1': '2'} AND durable_writes = true"
!docker exec -it cassandra cqlsh -e \
"use elk_c;CREATE TABLE employee(id int PRIMARY KEY,name text,city text,salary varint,phone varint)"

### data

In [4]:
employees_csv = open('cassandra_vol/employees.csv', 'w')

# int , text , text , varint, varint
# id ,  name , city , salary , phone
employees_csv.write('1,Adam,NOLA,60,5551234\n')
employees_csv.write('2,Bob,LA,70,3331234\n')
employees_csv.write('3,Chuck,NYC,80,6661234\n')
employees_csv.write('4,Dave,BOS,90,9991234\n')
employees_csv.write('5,Earl,RAL,100,1111234\n')

employees_csv.flush()
employees_csv.close()

!cat cassandra_vol/employees.csv

1,Adam,NOLA,60,5551234
2,Bob,LA,70,3331234
3,Chuck,NYC,80,6661234
4,Dave,BOS,90,9991234
5,Earl,RAL,100,1111234


### insert

In [5]:
!docker exec -it cassandra cqlsh -e \
"use elk_c; COPY employee(id,name,city,salary,phone) FROM '/ddl/employees.csv' WITH DELIMITER=','"

Using 9 child processes

Starting copy of elk_c.employee with columns [id, name, city, salary, phone].
Processed: 5 rows; Rate:      16 rows/s; Avg. rate:      16 rows/sProcessed: 5 rows; Rate:       8 rows/s; Avg. rate:      12 rows/s
5 rows imported from 1 files in 0.417 seconds (0 skipped).


### query

In [6]:
!docker exec -it cassandra cqlsh -e "use elk_c;select * from employee"


 id | city | name  | phone   | salary
----+------+-------+---------+--------
  5 |  RAL |  Earl | 1111234 |    100
  1 | NOLA |  Adam | 5551234 |     60
  2 |   LA |   Bob | 3331234 |     70
  4 |  BOS |  Dave | 9991234 |     90
  3 |  NYC | Chuck | 6661234 |     80

(5 rows)


<img src='imgs/elastic.png' width='550'/>

In [7]:
elastic_version    = '6.6.2'
h_elastic_rest_port  = '9200'
c_elastic_rest_port  = '9200'
h_elastic_node_comms = '9300'
c_elastic_node_comms = '9300'

!docker run -d                                                \
            -p {h_elastic_rest_port}:{c_elastic_rest_port}    \
            -p {h_elastic_node_comms}:{c_elastic_node_comms}  \
            -e "discovery.type=single-node"                   \
            -it                                               \
            -h elasticsearch                                  \
            --name elasticsearch                              \
            elasticsearch:{elastic_version}

3fb65600def4bc0655fd9e02e9bcf48e8a155bba9837b2fcaec888c03581e42e


In [8]:
!wget http://localhost:9200/ -q -O -

{
  "name" : "u8IFYNx",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "tPor0MsASaCoxLp31bJJuw",
  "version" : {
    "number" : "6.6.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "3bd3e59",
    "build_date" : "2019-03-06T15:16:26.864148Z",
    "build_snapshot" : false,
    "lucene_version" : "7.6.0",
    "minimum_wire_compatibility_version" : "5.6.0",
    "minimum_index_compatibility_version" : "5.0.0"
  },
  "tagline" : "You Know, for Search"
}


<img src='imgs/logstash.png' width='550'/>

### jdbc driver

In [9]:
!echo 'verify location:'
!ls -l   logstash_vol/cassandra-jdbc-driver-1.3.jar

!echo
!echo 'verify class name:'
!jar tvf logstash_vol/cassandra-jdbc-driver-1.3.jar | grep CassandraJdbcDriver

verify location:
-rw-r--r--  1 Bhill  staff  7645422 Apr 24 10:06 logstash_vol/cassandra-jdbc-driver-1.3.jar

verify class name:
  6211 Mon Feb 11 15:37:56 EST 2019 com/dbschema/CassandraJdbcDriver.class


### config

In [10]:
!cat logstash_vol/logstash.cassandra.conf

input {
  jdbc {
    jdbc_driver_library    => "/config-dir/cassandra-jdbc-driver-1.3.jar"
    jdbc_driver_class      => "com.dbschema.CassandraJdbcDriver"
    jdbc_connection_string => "jdbc:cassandra://cassandra:9042/elk_c"
    jdbc_user              => "root"
    statement              => "select id, city, name, cast(phone as int), cast(salary as int) from employee"
  }
}
output {  
    elasticsearch {  
	hosts => ["elasticsearch:9200"]  
	index => "employees"
    }  
}  


### load & quit ~ 40 seconds

In [11]:
!date
!time docker run -h logstash                       \
            --name logstash                        \
            --link cassandra:cassandra             \
            --link elasticsearch:elasticsearch     \
            -it --rm                               \
            -v ${PWD}/logstash_vol:/config-dir     \
            logstash:6.6.2                         \
            -f /config-dir/logstash.cassandra.conf > logstash.log
!tail -4 logstash.log                

Wed Apr 24 17:19:20 EDT 2019

real	0m41.231s
user	0m0.032s
sys	0m0.022s
[2019-04-24T21:19:56,517][INFO ][com.datastax.driver.core.Cluster] New Cassandra host cassandra/172.17.0.2:9042 added
[2019-04-24T21:19:56,862][INFO ][logstash.inputs.jdbc     ] (0.044362s) select id, city, name, cast(phone as int), cast(salary as int) from employee
[2019-04-24T21:19:58,719][INFO ][logstash.pipeline        ] Pipeline has terminated {:pipeline_id=>"main", :thread=>"#<Thread:0x6d1e781b run>"}
[2019-04-24T21:20:00,959][INFO ][logstash.pipeline        ] Pipeline has terminated {:pipeline_id=>".monitoring-logstash", :thread=>"#<Thread:0x38584267 run>"}


# query test

In [12]:
!wget http://localhost:9200/employees/_search\?pretty\=true\&q\=city:NOLA -q -O -

{
  "took" : 147,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.2876821,
    "hits" : [
      {
        "_index" : "employees",
        "_type" : "doc",
        "_id" : "rAs4UWoBQTO6dTyvL1pP",
        "_score" : 0.2876821,
        "_source" : {
          "id" : 1,
          "city" : "NOLA",
          "cast(phone as int)" : 5551234,
          "cast(salary as int)" : 60,
          "@version" : "1",
          "name" : "Adam",
          "@timestamp" : "2019-04-24T21:19:56.946Z"
        }
      }
    ]
  }
}


<img src='imgs/kibana.png' width='450' />

In [13]:
h_kibana_port        = '5601'
c_kibana_port        = '5601'
!docker run -d                                  \
            -p {h_kibana_port}:{c_kibana_port}  \
            -h kibana                           \
            --name kibana                       \
            --link elasticsearch:elasticsearch  \
            kibana:{elastic_version}

6fad332bc69d4746c672f35b3cbdbb7576822d388bf75535f1a4596064930ba7


http://localhost:5601

<img src='imgs/kibana_timelion.png' width='350' />

# stop & kill

In [14]:
!docker stop cassandra     && docker rm cassandra 
!docker stop kibana        && docker rm kibana
!docker stop elasticsearch && docker rm elasticsearch
!docker stop logstash      && docker rm logstash

cassandra
cassandra
kibana
kibana
elasticsearch
elasticsearch
Error response from daemon: No such container: logstash


In [15]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


<img src='imgs/docker_compose.png' width='550' />

# docker compose version

In [16]:
!cat docker-compose.yml

version: "3"
services:
  cassandra:
    image: "cassandra:3.11"
    hostname: "cassandra"
    container_name: "cassandra"
    ports:
      - "7000:7000"
      - "9042:9042"
      - "9160:9160"
    volumes:
      - "${PWD}/cassandra_vol:/ddl"
  elasticsearch:
    image: "elasticsearch:6.6.2"
    hostname: "elasticsearch"
    container_name: elasticsearch
    stdin_open: true
    tty: true
    ports:
      - "9200:9200"
      - "9300:9300"
    environment:
      - "discovery.type=single-node"
  kibana:
    image: "kibana:6.6.2"
    hostname: "kibana"
    container_name: "kibana"
    ports:
      - "5601:5601"
    links:
      - "elasticsearch:elasticsearch"
  logstash:
    image: "logstash:6.6.2"
    hostname: "logstash"
    container_name: "logstash"
    stdin_open: true
    tty: true
    volumes:
      - "${PWD}/logstash_vol:/config-dir"
    links:
      - "cassandra:cassandra"
      - "elasticsearch:elasticsearch"

In [ ]:
# !docker-compose up -d